In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
from keras import regularizers

In [3]:
import os

In [4]:
# -*- coding: utf-8 -*-  
import os  
  
def listdir(path,list_name):  
#     list_name=list()
    label_list=list()
    for file in os.listdir(path):  
        file_path = os.path.join(path, file)  
        if os.path.isdir(file_path):  
            listdir(file_path, list_name)  
        elif os.path.splitext(file_path)[1]=='.wav':  
            list_name.append(file_path)

In [5]:
list_name=list()
label_list=listdir('RawData/CASIA database',list_name)

In [6]:
len(list_name)

1200

In [7]:
list_name

['RawData/CASIA database\\liuchanhg\\angry\\201.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\202.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\203.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\204.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\205.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\206.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\207.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\208.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\209.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\210.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\211.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\212.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\213.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\214.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\215.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\216.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\217.wav',
 'RawData/CASIA database\\liuchanhg\\angry\\218.wav',
 'RawData/CASIA database\\li

In [8]:
# listdir

In [9]:
# list_name

In [10]:
type(list_name[0])

str

In [11]:
# name = "mkyong is learning python 123"
name=list_name[0]
label_list=list()
if "angry" in name:
    print("found python!")
    label_list.append(0)
else:
    print("nothing")

found python!


In [12]:
label_list

[0]

In [13]:
list_name[0][:]

'RawData/CASIA database\\liuchanhg\\angry\\201.wav'

In [14]:
# name_list=["angry","fear","happy","neutral","sad","surprise"]

In [16]:
label_list=list()
for i in range(len(list_name)):
#     print(list_name)
    name=list_name[i]
    if "liuchanhg" in name:
        label_list.append("liuchanhg")
    if "wangzhe" in name:
        label_list.append("wangzhe")
    if "zhaoquanyin" in name:
        label_list.append("zhaoquanyin")
    if "ZhaoZuoxiang" in name:
        label_list.append("ZhaoZuoxiang")

In [17]:
len(label_list)

1200

In [18]:
label_list

['liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuchanhg',
 'liuc

In [19]:
import pandas as pd

In [20]:
labels = pd.DataFrame(label_list)

In [21]:
labels[:10]

0
0  liuchanhg
1  liuchanhg
2  liuchanhg
3  liuchanhg
4  liuchanhg
5  liuchanhg
6  liuchanhg
7  liuchanhg
8  liuchanhg
9  liuchanhg

In [22]:
labels.columns

RangeIndex(start=0, stop=1, step=1)

In [23]:
# label_list

In [24]:
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(list_name):
#         print(y)
        X, sample_rate = librosa.load(y, res_type='kaiser_fast'
                                      ,duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                        axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1  

In [25]:
df[:5]

feature
0  [-6.6186733, -8.488445, -4.872398, -2.9646916,...
1  [0.4991513, -1.5759646, -6.207563, -10.585773,...
2  [-35.05051, -34.26611, -35.925133, -35.464096,...
3  [-7.753039, -9.13146, -9.178799, -6.578576, -6...
4  [-12.166086, -11.5131855, -12.888262, -15.0473...

In [26]:
# df[:1].feature

In [27]:
df3 = pd.DataFrame(df['feature'].values.tolist())

In [28]:
newdf = pd.concat([df3,labels], axis=1)

In [29]:
rnewdf = newdf.rename(index=str, columns={"0": "label"})

In [30]:
rnewdf[:5]

0          1          2          3          4          5    \
0  -6.618673  -8.488445  -4.872398  -2.964692  -8.330674 -12.109244   
1   0.499151  -1.575965  -6.207563 -10.585773 -20.253441 -23.829960   
2 -35.050510 -34.266109 -35.925133 -35.464096 -34.905174 -34.601162   
3  -7.753039  -9.131460  -9.178799  -6.578576  -6.647866 -10.657045   
4 -12.166086 -11.513186 -12.888262 -15.047313 -12.117293  -8.700770   

         6          7          8          9    ...  207  208  209  210  211  \
0 -16.261909 -18.448881 -20.559122 -24.176138  ...  NaN  NaN  NaN  NaN  NaN   
1 -22.893242 -19.042793 -15.880293 -16.978241  ...  NaN  NaN  NaN  NaN  NaN   
2 -34.526993 -35.273239 -36.000118 -31.690880  ...  NaN  NaN  NaN  NaN  NaN   
3 -12.422474 -14.158489 -12.462173 -13.015013  ...  NaN  NaN  NaN  NaN  NaN   
4 -10.092604 -15.213968 -18.787750 -20.347828  ...  NaN  NaN  NaN  NaN  NaN   

   212  213  214  215        0    
0  NaN  NaN  NaN  NaN  liuchanhg  
1  NaN  NaN  NaN  NaN  liuchanhg  
2  NaN  NaN  NaN  NaN  liuchanhg  
3  NaN  NaN  NaN  NaN  liuchanhg  
4  NaN  NaN  NaN  NaN  liuchanhg  

[5 rows x 217 columns]

In [31]:
from sklearn.utils import shuffle
rnewdf = shuffle(newdf)
rnewdf[:10]

0          1          2          3          4          5    \
413   -8.599478 -10.547052 -13.313433 -13.642029 -13.971526 -14.108351   
149   -3.407436  -8.192620 -33.316586 -32.651981 -34.194565 -32.263916   
433  -10.076146 -11.291896 -16.313286 -16.496819 -18.295849 -18.849760   
534  -11.375294 -11.347844 -15.781165 -20.971340 -18.381023 -15.731955   
1059  -5.146158  -6.532760  -7.775607  -7.725957  -7.604078  -7.627913   
1128 -34.800636 -34.885815 -35.645344 -35.644535 -35.738728 -36.171299   
1198 -34.178123 -34.027969 -36.268330 -35.008389 -34.760921 -34.028828   
1157 -34.424725 -34.530849 -35.561581 -35.890926 -35.433567 -32.405983   
440  -12.139663 -12.011146 -14.177792 -16.601772 -18.537451 -18.697762   
86     2.190799  -1.042535 -12.580463 -15.167750 -15.359455 -14.743037   

            6          7          8          9    ...        207        208  \
413  -14.582280 -14.346706 -14.568965 -14.398244  ...        NaN        NaN   
149  -31.983355 -35.592567 -37.135925 -37.728706  ...        NaN        NaN   
433  -20.498310 -20.380014 -22.602978 -20.802355  ...        NaN        NaN   
534  -11.175664  -5.800423  -4.533355  -3.493218  ...        NaN        NaN   
1059  -7.823384  -8.119743  -8.404648  -8.816866  ...        NaN        NaN   
1128 -36.005909 -35.834763 -35.242325 -36.398502  ... -13.756220 -10.955211   
1198 -33.903515 -34.987617 -33.466133 -32.469387  ... -33.637905 -34.769650   
1157 -29.752010 -25.405638 -24.996708 -28.727486  ... -36.953945 -33.572762   
440  -18.095133 -20.432133 -27.135826 -31.941132  ...        NaN        NaN   
86   -15.747364 -16.346188 -16.662691 -15.038390  ...        NaN        NaN   

            209        210       211        212        213        214  \
413         NaN        NaN       NaN        NaN        NaN        NaN   
149         NaN        NaN       NaN        NaN        NaN        NaN   
433         NaN        NaN       NaN        NaN        NaN        NaN   
534         NaN        NaN       NaN        NaN        NaN        NaN   
1059        NaN        NaN       NaN        NaN        NaN        NaN   
1128 -10.956670 -11.586684 -12.76188 -13.047077 -14.291899  -9.338861   
1198 -34.488956 -33.853329 -35.27161 -34.852501 -33.892818 -33.959667   
1157        NaN        NaN       NaN        NaN        NaN        NaN   
440         NaN        NaN       NaN        NaN        NaN        NaN   
86          NaN        NaN       NaN        NaN        NaN        NaN   

            215           0    
413         NaN       wangzhe  
149         NaN     liuchanhg  
433         NaN       wangzhe  
534         NaN       wangzhe  
1059        NaN  ZhaoZuoxiang  
1128  -6.255818  ZhaoZuoxiang  
1198 -35.638138  ZhaoZuoxiang  
1157        NaN  ZhaoZuoxiang  
440         NaN       wangzhe  
86          NaN     liuchanhg  

[10 rows x 217 columns]

In [32]:
rnewdf=rnewdf.fillna(0)

In [33]:
newdf1 = np.random.rand(len(rnewdf)) < 0.8
train = rnewdf[newdf1]
test = rnewdf[~newdf1]

In [34]:
train[250:260]

0          1          2          3          4          5    \
567   -7.702151  -8.338616  -8.323007   0.479526   3.066850   0.315488   
1132  -6.523319  -8.282507 -12.276844 -12.744915 -13.544188 -15.369818   
168  -16.043581 -19.205284 -18.322124 -10.098906 -10.312380 -22.482801   
518   -1.307962  -4.240071 -12.578726 -18.189528 -25.356245 -25.978981   
742  -17.443302 -19.734467 -21.426008 -23.843819 -25.025442 -24.500307   
252  -13.273704 -17.457150 -27.819057 -25.414385 -18.743477 -12.432072   
830  -27.720263 -26.084208 -22.947029 -19.478825 -18.206297 -18.340975   
917   -3.684708  -4.065282  -8.253188  -8.275100 -10.092931 -14.958588   
437   -8.569131 -11.740641 -19.084148 -17.054033 -17.775528 -18.933918   
719   -9.461572 -13.339234 -25.802151 -24.830383 -28.156246 -33.493298   

            6          7          8          9    ...  207  208  209  210  \
567   -4.270428  -5.609313  -5.116682  -4.127511  ...  0.0  0.0  0.0  0.0   
1132 -19.494463 -21.311724 -20.522524 -18.955715  ...  0.0  0.0  0.0  0.0   
168  -29.190382 -14.693003  -0.490522   2.918244  ...  0.0  0.0  0.0  0.0   
518  -27.367317 -28.872374 -24.411924 -19.944145  ...  0.0  0.0  0.0  0.0   
742  -21.572958 -15.720302 -16.133617 -19.812611  ...  0.0  0.0  0.0  0.0   
252  -11.539265  -7.372513   1.365557   5.888610  ...  0.0  0.0  0.0  0.0   
830  -18.032724 -19.598459 -23.282871 -25.051880  ...  0.0  0.0  0.0  0.0   
917  -18.301399 -19.880627 -22.390577 -25.057945  ...  0.0  0.0  0.0  0.0   
437  -20.165619 -19.863142 -17.929323 -19.907223  ...  0.0  0.0  0.0  0.0   
719  -19.521341 -13.423841 -14.990063 -21.726866  ...  0.0  0.0  0.0  0.0   

      211  212  213  214  215           0    
567   0.0  0.0  0.0  0.0  0.0       wangzhe  
1132  0.0  0.0  0.0  0.0  0.0  ZhaoZuoxiang  
168   0.0  0.0  0.0  0.0  0.0     liuchanhg  
518   0.0  0.0  0.0  0.0  0.0       wangzhe  
742   0.0  0.0  0.0  0.0  0.0   zhaoquanyin  
252   0.0  0.0  0.0  0.0  0.0     liuchanhg  
830   0.0  0.0  0.0  0.0  0.0   zhaoquanyin  
917   0.0  0.0  0.0  0.0  0.0  ZhaoZuoxiang  
437   0.0  0.0  0.0  0.0  0.0       wangzhe  
719   0.0  0.0  0.0  0.0  0.0   zhaoquanyin  

[10 rows x 217 columns]

In [35]:
trainfeatures = train.iloc[:, :-1]

In [36]:
trainlabel = train.iloc[:, -1:]

In [37]:
testfeatures = test.iloc[:, :-1]

In [38]:
testlabel = test.iloc[:, -1:]

In [39]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [40]:
y_train

array([[0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [41]:
X_train.shape

(955, 216)

In [42]:
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

In [43]:
model = Sequential()

model.add(Conv1D(256, 5,padding='same',
                 input_shape=(216,1)))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(4))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)

In [44]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 216, 256)          1536      
_________________________________________________________________
activation_1 (Activation)    (None, 216, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 216, 128)          163968    
_________________________________________________________________
activation_2 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 216, 128)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 27, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 27, 128)          

In [45]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [46]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=128, epochs=700, validation_data=(x_testcnn, y_test))

Train on 955 samples, validate on 245 samples
Epoch 1/700
955/955 [==============================] - 8s 9ms/step - loss: 1.4192 - accuracy: 0.2880 - val_loss: 1.3690 - val_accuracy: 0.3551
Epoch 2/700
955/955 [==============================] - 7s 8ms/step - loss: 1.3284 - accuracy: 0.3340 - val_loss: 1.3244 - val_accuracy: 0.3755
Epoch 3/700
955/955 [==============================] - 7s 8ms/step - loss: 1.2833 - accuracy: 0.3759 - val_loss: 1.2929 - val_accuracy: 0.3837
Epoch 4/700
955/955 [==============================] - 7s 8ms/step - loss: 1.2459 - accuracy: 0.4147 - val_loss: 1.2635 - val_accuracy: 0.4041
Epoch 5/700
955/955 [==============================] - 8s 8ms/step - loss: 1.2159 - accuracy: 0.4377 - val_loss: 1.2399 - val_accuracy: 0.4082
Epoch 6/700
955/955 [==============================] - 8s 8ms/step - loss: 1.1853 - accuracy: 0.4628 - val_loss: 1.2163 - val_accuracy: 0.4286
Epoch 7/700
955/955 [==============================] - 8s 8ms/step - loss: 1.1601 - accuracy: 0.

955/955 [==============================] - 8s 8ms/step - loss: 0.7800 - accuracy: 0.7183 - val_loss: 0.8863 - val_accuracy: 0.6327
Epoch 58/700
955/955 [==============================] - 8s 8ms/step - loss: 0.7777 - accuracy: 0.7037 - val_loss: 0.8887 - val_accuracy: 0.6531
Epoch 59/700
955/955 [==============================] - 7s 8ms/step - loss: 0.7770 - accuracy: 0.7141 - val_loss: 0.8808 - val_accuracy: 0.6408
Epoch 60/700
955/955 [==============================] - 8s 8ms/step - loss: 0.7702 - accuracy: 0.7204 - val_loss: 0.8841 - val_accuracy: 0.6449
Epoch 61/700
955/955 [==============================] - 10s 10ms/step - loss: 0.7688 - accuracy: 0.7246 - val_loss: 0.8850 - val_accuracy: 0.6449
Epoch 62/700
955/955 [==============================] - 8s 8ms/step - loss: 0.7665 - accuracy: 0.7236 - val_loss: 0.8729 - val_accuracy: 0.6490
Epoch 63/700
955/955 [==============================] - 7s 7ms/step - loss: 0.7611 - accuracy: 0.7194 - val_loss: 0.8766 - val_accuracy: 0.6612
Epo

955/955 [==============================] - 8s 8ms/step - loss: 0.6412 - accuracy: 0.7770 - val_loss: 0.7716 - val_accuracy: 0.7102
Epoch 114/700
955/955 [==============================] - 7s 8ms/step - loss: 0.6374 - accuracy: 0.7717 - val_loss: 0.7703 - val_accuracy: 0.7102
Epoch 115/700
955/955 [==============================] - 8s 8ms/step - loss: 0.6374 - accuracy: 0.7801 - val_loss: 0.7714 - val_accuracy: 0.7265
Epoch 116/700
955/955 [==============================] - 8s 8ms/step - loss: 0.6339 - accuracy: 0.7822 - val_loss: 0.7651 - val_accuracy: 0.7020
Epoch 117/700
955/955 [==============================] - 8s 8ms/step - loss: 0.6291 - accuracy: 0.7853 - val_loss: 0.7723 - val_accuracy: 0.7143
Epoch 118/700
955/955 [==============================] - 7s 8ms/step - loss: 0.6290 - accuracy: 0.7864 - val_loss: 0.7668 - val_accuracy: 0.7184
Epoch 119/700
955/955 [==============================] - 8s 8ms/step - loss: 0.6290 - accuracy: 0.7853 - val_loss: 0.7635 - val_accuracy: 0.7143

955/955 [==============================] - 8s 9ms/step - loss: 0.5426 - accuracy: 0.8147 - val_loss: 0.6909 - val_accuracy: 0.7510
Epoch 170/700
955/955 [==============================] - 8s 9ms/step - loss: 0.5395 - accuracy: 0.8168 - val_loss: 0.6944 - val_accuracy: 0.7633
Epoch 171/700
955/955 [==============================] - 8s 8ms/step - loss: 0.5370 - accuracy: 0.8199 - val_loss: 0.6979 - val_accuracy: 0.7510
Epoch 172/700
955/955 [==============================] - 8s 8ms/step - loss: 0.5377 - accuracy: 0.8188 - val_loss: 0.6970 - val_accuracy: 0.7551
Epoch 173/700
955/955 [==============================] - 7s 8ms/step - loss: 0.5351 - accuracy: 0.8157 - val_loss: 0.6974 - val_accuracy: 0.7592
Epoch 174/700
955/955 [==============================] - 7s 8ms/step - loss: 0.5324 - accuracy: 0.8230 - val_loss: 0.6869 - val_accuracy: 0.7551
Epoch 175/700
955/955 [==============================] - 8s 8ms/step - loss: 0.5301 - accuracy: 0.8178 - val_loss: 0.6817 - val_accuracy: 0.7633

955/955 [==============================] - 10s 11ms/step - loss: 0.4651 - accuracy: 0.8419 - val_loss: 0.6389 - val_accuracy: 0.7673
Epoch 226/700
955/955 [==============================] - 11s 11ms/step - loss: 0.4601 - accuracy: 0.8440 - val_loss: 0.6332 - val_accuracy: 0.7796
Epoch 227/700
955/955 [==============================] - 10s 11ms/step - loss: 0.4562 - accuracy: 0.8471 - val_loss: 0.6310 - val_accuracy: 0.7755
Epoch 228/700
955/955 [==============================] - 10s 11ms/step - loss: 0.4565 - accuracy: 0.8429 - val_loss: 0.6362 - val_accuracy: 0.7714
Epoch 229/700
955/955 [==============================] - 10s 10ms/step - loss: 0.4568 - accuracy: 0.8429 - val_loss: 0.6320 - val_accuracy: 0.7755
Epoch 230/700
955/955 [==============================] - 10s 10ms/step - loss: 0.4564 - accuracy: 0.8440 - val_loss: 0.6258 - val_accuracy: 0.7714
Epoch 231/700
955/955 [==============================] - 10s 11ms/step - loss: 0.4519 - accuracy: 0.8513 - val_loss: 0.6300 - val_ac

...

955/955 [==============================] - 8s 9ms/step - loss: 0.2525 - accuracy: 0.9257 - val_loss: 0.5303 - val_accuracy: 0.7837
Epoch 450/700
955/955 [==============================] - 8s 9ms/step - loss: 0.2537 - accuracy: 0.9257 - val_loss: 0.5365 - val_accuracy: 0.7837
Epoch 451/700
955/955 [==============================] - 8s 8ms/step - loss: 0.2486 - accuracy: 0.9298 - val_loss: 0.5579 - val_accuracy: 0.7755
Epoch 452/700
955/955 [==============================] - 8s 8ms/step - loss: 0.2489 - accuracy: 0.9277 - val_loss: 0.5216 - val_accuracy: 0.8122
Epoch 453/700
955/955 [==============================] - 8s 8ms/step - loss: 0.2561 - accuracy: 0.9309 - val_loss: 0.5300 - val_accuracy: 0.7878
Epoch 454/700
955/955 [==============================] - 7s 8ms/step - loss: 0.2542 - accuracy: 0.9277 - val_loss: 0.5399 - val_accuracy: 0.7837
Epoch 455/700
955/955 [==============================] - 7s 8ms/step - loss: 0.2533 - accuracy: 0.9215 - val_loss: 0.5213 - val_accuracy: 0.7918

955/955 [==============================] - 7s 8ms/step - loss: 0.2160 - accuracy: 0.9414 - val_loss: 0.5160 - val_accuracy: 0.7959
Epoch 506/700
955/955 [==============================] - 7s 8ms/step - loss: 0.2174 - accuracy: 0.9424 - val_loss: 0.5187 - val_accuracy: 0.7918
Epoch 507/700
955/955 [==============================] - 8s 8ms/step - loss: 0.2132 - accuracy: 0.9372 - val_loss: 0.5173 - val_accuracy: 0.7918
Epoch 508/700
955/955 [==============================] - 7s 8ms/step - loss: 0.2147 - accuracy: 0.9414 - val_loss: 0.5207 - val_accuracy: 0.8122
Epoch 509/700
955/955 [==============================] - 8s 8ms/step - loss: 0.2163 - accuracy: 0.9361 - val_loss: 0.5379 - val_accuracy: 0.7633
Epoch 510/700
955/955 [==============================] - 8s 9ms/step - loss: 0.2074 - accuracy: 0.9414 - val_loss: 0.5220 - val_accuracy: 0.7796
Epoch 511/700
955/955 [==============================] - 8s 8ms/step - loss: 0.2142 - accuracy: 0.9424 - val_loss: 0.5122 - val_accuracy: 0.7918

955/955 [==============================] - 9s 9ms/step - loss: 0.1827 - accuracy: 0.9592 - val_loss: 0.5189 - val_accuracy: 0.8041
Epoch 562/700
955/955 [==============================] - 9s 10ms/step - loss: 0.1848 - accuracy: 0.9550 - val_loss: 0.5315 - val_accuracy: 0.7796
Epoch 563/700
955/955 [==============================] - 10s 11ms/step - loss: 0.1811 - accuracy: 0.9581 - val_loss: 0.5178 - val_accuracy: 0.7796
Epoch 564/700
955/955 [==============================] - 8s 8ms/step - loss: 0.1776 - accuracy: 0.9529 - val_loss: 0.5342 - val_accuracy: 0.7755
Epoch 565/700
955/955 [==============================] - 7s 8ms/step - loss: 0.1766 - accuracy: 0.9539 - val_loss: 0.5186 - val_accuracy: 0.7755
Epoch 566/700
955/955 [==============================] - 7s 8ms/step - loss: 0.1828 - accuracy: 0.9529 - val_loss: 0.5116 - val_accuracy: 0.8000
Epoch 567/700
955/955 [==============================] - 7s 8ms/step - loss: 0.1762 - accuracy: 0.9613 - val_loss: 0.5189 - val_accuracy: 0.7

955/955 [==============================] - 9s 10ms/step - loss: 0.1549 - accuracy: 0.9602 - val_loss: 0.5132 - val_accuracy: 0.8041
Epoch 618/700
955/955 [==============================] - 9s 9ms/step - loss: 0.1547 - accuracy: 0.9686 - val_loss: 0.5122 - val_accuracy: 0.7878
Epoch 619/700
955/955 [==============================] - 8s 9ms/step - loss: 0.1496 - accuracy: 0.9696 - val_loss: 0.5198 - val_accuracy: 0.7755
Epoch 620/700
955/955 [==============================] - 8s 9ms/step - loss: 0.1547 - accuracy: 0.9613 - val_loss: 0.5160 - val_accuracy: 0.7796
Epoch 621/700
955/955 [==============================] - 8s 8ms/step - loss: 0.1481 - accuracy: 0.9675 - val_loss: 0.5334 - val_accuracy: 0.7878
Epoch 622/700
955/955 [==============================] - 8s 9ms/step - loss: 0.1497 - accuracy: 0.9623 - val_loss: 0.5199 - val_accuracy: 0.7714
Epoch 623/700
955/955 [==============================] - 8s 8ms/step - loss: 0.1477 - accuracy: 0.9634 - val_loss: 0.5219 - val_accuracy: 0.775

955/955 [==============================] - 9s 9ms/step - loss: 0.1267 - accuracy: 0.9749 - val_loss: 0.5353 - val_accuracy: 0.8000
Epoch 674/700
955/955 [==============================] - 9s 9ms/step - loss: 0.1334 - accuracy: 0.9717 - val_loss: 0.5121 - val_accuracy: 0.7837
Epoch 675/700
955/955 [==============================] - 10s 10ms/step - loss: 0.1272 - accuracy: 0.9749 - val_loss: 0.5198 - val_accuracy: 0.7714
Epoch 676/700
955/955 [==============================] - 8s 8ms/step - loss: 0.1225 - accuracy: 0.9759 - val_loss: 0.5545 - val_accuracy: 0.7755
Epoch 677/700
955/955 [==============================] - 8s 8ms/step - loss: 0.1236 - accuracy: 0.9728 - val_loss: 0.5221 - val_accuracy: 0.7755
Epoch 678/700
955/955 [==============================] - 9s 9ms/step - loss: 0.1286 - accuracy: 0.9738 - val_loss: 0.5121 - val_accuracy: 0.7918
Epoch 679/700
955/955 [==============================] - 9s 10ms/step - loss: 0.1203 - accuracy: 0.9759 - val_loss: 0.5136 - val_accuracy: 0.7

In [47]:
# plt.plot(cnnhistory.history['loss'])
# plt.plot(cnnhistory.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [48]:
model_name = 'Emotion_Voice_Detection_People_Model20201122.h5'
save_dir = os.path.join(os.getcwd(), 'saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at H:\textCnnTest\英俊的全部能力\实战功法\语音识别\Speech-Emotion-Analyzer\saved_models\Emotion_Voice_Detection_People_Model20201122.h5 


In [49]:
import json
model_json = model.to_json()
with open("model_People_20201122.json", "w") as json_file:
    json_file.write(model_json)

In [50]:
# loading json and creating model
from keras.models import model_from_json
json_file = open('model_People_20201122.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("saved_models/Emotion_Voice_Detection_People_Model20201122.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
score = loaded_model.evaluate(x_testcnn, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 80.00%


In [51]:
preds = loaded_model.predict(x_testcnn, 
                         batch_size=32, 
                         verbose=1)

245/245 [==============================] - 0s 2ms/step


In [52]:
preds

array([[1.11828430e-03, 3.92802842e-02, 8.66422057e-01, 9.31793302e-02],
       [6.35011401e-03, 1.58954412e-03, 9.70695496e-01, 2.13648584e-02],
       [3.07804323e-04, 9.70706761e-01, 7.02128559e-03, 2.19642241e-02],
       [2.10479615e-04, 4.91896793e-02, 1.50991017e-02, 9.35500741e-01],
       [2.79649132e-04, 7.14733847e-04, 9.98956800e-01, 4.88284750e-05],
       [1.38485164e-03, 1.91095732e-02, 9.79026794e-01, 4.78701259e-04],
       [1.45466311e-03, 6.88779712e-01, 3.53964511e-03, 3.06226075e-01],
       [4.61944155e-02, 1.48794159e-01, 7.96396315e-01, 8.61511752e-03],
       [1.72687825e-04, 2.19383240e-02, 1.08998276e-01, 8.68890703e-01],
       [6.57901037e-05, 9.97023761e-01, 3.84918366e-07, 2.90999515e-03],
       [9.83134261e-04, 1.16808176e-01, 6.89913988e-01, 1.92294747e-01],
       [9.99978781e-01, 2.02252686e-05, 1.38168726e-14, 9.76326078e-07],
       [9.99997973e-01, 1.92573816e-06, 4.96315896e-20, 1.28353108e-07],
       [1.37726788e-03, 5.78011274e-02, 8.35523784e

In [53]:
preds1=preds.argmax(axis=1)

In [54]:
preds1

array([2, 2, 1, 3, 2, 2, 1, 2, 3, 1, 2, 0, 0, 2, 2, 2, 0, 3, 1, 0, 1, 1,
       0, 1, 3, 2, 3, 2, 1, 1, 3, 2, 0, 3, 3, 0, 3, 1, 3, 1, 2, 2, 3, 3,
       3, 3, 1, 0, 0, 0, 3, 3, 0, 2, 3, 2, 2, 1, 3, 1, 3, 3, 0, 1, 0, 2,
       3, 1, 3, 0, 2, 2, 3, 2, 2, 3, 1, 2, 0, 2, 1, 0, 0, 0, 2, 2, 2, 3,
       0, 2, 0, 3, 3, 0, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 3, 1, 0, 3, 0, 1,
       1, 2, 3, 2, 3, 2, 1, 2, 1, 1, 2, 0, 0, 3, 0, 2, 2, 0, 1, 3, 0, 3,
       1, 3, 1, 0, 1, 1, 1, 3, 1, 3, 1, 3, 3, 1, 2, 0, 2, 0, 3, 2, 3, 1,
       0, 2, 1, 1, 3, 1, 0, 3, 1, 2, 1, 3, 1, 0, 2, 3, 3, 0, 0, 2, 1, 2,
       2, 0, 3, 0, 3, 0, 2, 1, 3, 2, 1, 0, 1, 2, 3, 3, 2, 3, 1, 2, 3, 0,
       0, 1, 3, 2, 3, 2, 2, 2, 1, 2, 0, 2, 3, 3, 2, 2, 2, 2, 0, 3, 1, 1,
       3, 3, 0, 0, 3, 3, 0, 0, 0, 1, 2, 2, 3, 0, 2, 0, 0, 0, 2, 1, 0, 3,
       2, 0, 3], dtype=int64)

In [55]:
abc = preds1.astype(int).flatten()

In [56]:
predictions = (lb.inverse_transform((abc)))

In [57]:
preddf = pd.DataFrame({'predictedvalues': predictions})
preddf[:10]

predictedvalues
0         wangzhe
1         wangzhe
2       liuchanhg
3     zhaoquanyin
4         wangzhe
5         wangzhe
6       liuchanhg
7         wangzhe
8     zhaoquanyin
9       liuchanhg

In [58]:
actual=y_test.argmax(axis=1)
abc123 = actual.astype(int).flatten()
actualvalues = (lb.inverse_transform((abc123)))

In [59]:
actualdf = pd.DataFrame({'actualvalues': actualvalues})
actualdf[:10]

actualvalues
0      wangzhe
1      wangzhe
2    liuchanhg
3  zhaoquanyin
4      wangzhe
5      wangzhe
6    liuchanhg
7      wangzhe
8  zhaoquanyin
9    liuchanhg

In [60]:
finaldf = actualdf.join(preddf)

In [61]:
finaldf[170:180]

actualvalues predictedvalues
170   zhaoquanyin     zhaoquanyin
171  ZhaoZuoxiang    ZhaoZuoxiang
172  ZhaoZuoxiang    ZhaoZuoxiang
173     liuchanhg         wangzhe
174       wangzhe       liuchanhg
175       wangzhe         wangzhe
176       wangzhe         wangzhe
177  ZhaoZuoxiang    ZhaoZuoxiang
178   zhaoquanyin     zhaoquanyin
179  ZhaoZuoxiang    ZhaoZuoxiang

In [62]:
finaldf.groupby('actualvalues').count()

predictedvalues
actualvalues                 
ZhaoZuoxiang               61
liuchanhg                  64
wangzhe                    65
zhaoquanyin                55

In [63]:
finaldf.groupby('predictedvalues').count()

actualvalues
predictedvalues              
ZhaoZuoxiang               55
liuchanhg                  56
wangzhe                    69
zhaoquanyin                65

In [64]:
finaldf.to_csv('PredictionsPeople20201122.csv', index=False)

In [65]:
# data, sampling_rate = librosa.load('output1020201122.wav')

In [66]:
# % pylab inline
# import os
# import pandas as pd
# import librosa
# import glob 

# plt.figure(figsize=(15, 5))
# librosa.display.waveplot(data, sr=sampling_rate)

In [67]:
import gc

In [68]:
gc.collect()

19804